# Neural Networks Optimized by Quantum Genetic Algorithm

Usually, feedforward neural networks are optimized by backpropagation method, to get the derivatives of weights and biases, and optimize them during each iteration of stochastic gradient descending. However, sometimes it is difficult to get the derivatives of the cost functions and sometimes the cost functions are not the same with the metrics. There are alternatives for backpropagation methods, such as genetic algorithms which can search for optimal parameters using iterations of populations and generations.

The conventional genetic algorithm can generate a large population of specified neural networks with different parameters, each neural network with a specific parameter combinations is an individual. Then through calculation of fitnesses, cross-over, mutations, the algorithm tries to evolve the population in terms of the fitness.

An important factor of applying genetic algorithm to real-world problem is encoding, namely encoding the key factors into genes, such as binary coding, namely encode a parameter as a series of 0s or 1s.  However, In this project, we are going to encode the weights and biases with quantums. The encoding method refers to Dr. Wang Zhiteng's [paper](https://wenku.baidu.com/view/8fe5bc385a8102d276a22f8b.html).

In [2]:
import numpy as np
import math
from time import time
import matplotlib.pyplot as plt
%matplotlib inline

In order to validate the performances of Quantum Genetic ALgorithm and compare it to classical machine learning algorithms such as fully-connected neural networks, we use Iris dataset and MNIST to do training and testing.

In [352]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
iris = datasets.load_iris()

In [28]:
iris.data.shape

(150, 4)

In [29]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'],
      dtype='<U10')

In [232]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=1)

# Backpropagation Model

MNIST is a very classical image dataset for classification, first we can build a neural network by sklearn, and the parameters will be updated by backpropogation method.

In [303]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(20,), 
                    activation='logistic', verbose=0, max_iter=100)

In [304]:
mlp.fit(X_train, y_train)

C:\Program Files\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(20,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=100, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=0, warm_start=False)

In [305]:
preds = mlp.predict(X_train)
print('Training Accuracy', round(np.mean(preds == y_train), 3))

Training Accuracy 0.692


In [306]:
preds = mlp.predict(X_test)
print('Testing Accuracy', round(np.mean(preds == y_test), 3))

Testing Accuracy 0.567


In [307]:
y_test

array([0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 2, 1, 0, 0, 1, 2])

It is pretty easy to design and optimize a neural network by tools like sklearn or tensorflow based on backpropagation method, but note, **the cost function must be differentiable in order to get gradients and the metric may not be the same with the cost function**. For example, we can use cross-entropy as our cost function for classification problems, however we often use accuracy as the metrics which is more explicit although not differentiable. 

Next, we are going to propose a quantum genetic algorithm, which does not rely on differentiable cost functions, you can define any cost functions. In addition, the mechanism of genetic algorithm can make use of distributed computing as much as possible.

# 1.Feedforward Neural Network

We need to compute feedforward propagation given input and weights, it is quite easy to initialize them. And with different parameters, we can initialize different neural networks.
## Qubit
A qubit is a pair of $\alpha=sin(\theta)$ and $\beta=cos(\theta)$, which shows the uncertainties of a state.

## Qubit Encoding
In a quantum genetic algorithm, each chromosom consists of a series of qubits that forms a quantum registra. For example, suppose we have $m$ parameters, and each parameter can be encoded as a 5-qubit series, consequently the length of the chromosome should be $5m$. 

For example, we can denote the $ith$ parameter $x_i$:
$$\left(\begin{array}{ccccc}
		\alpha_{i1} & \alpha_{i2} & \alpha_{i3} & \alpha_{i4} & \alpha_{i5}\\ 
		\beta_{i1} & \beta_{i2} & \beta_{i3} & \beta_{i4} & \beta_{i5}
	\end{array}\right)$$
    
 ## Collapsing
 We can map the qubits series above into states(5 binaries) according to certain criteria(will discuss later), for example we can suppose the collapsed states of the qubits above as:
 $$s_i = [0 \ 1 \ 1\ 0 \ 0]$$
 
 ## Value Calculating
 Now that we have the collapsed states of a parameter $x_i$, we can calculate the decimal value of it with a specified range:
 $$x_i = lowerbound + \frac {\sum_{j=1}^5 s_{ij} 2^{j-1}} {2^5-1} * (upperbound- lowerbound)$$

We initialize weights and biases according to uniform distribution.

In [283]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [39]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
#y_train = enc.fit_transform(y_train.reshape(-1, 1))
#y_test = enc.transform(y_test.reshape(-1, 1))

In [66]:
qubit_len = 10
def initializeWeights(layers):
    if type(layers) is not list:
        print('Wrong input!')
        return None
    layers_num = len(layers)
    weights = []
    for l in range(layers_num-1):
        #Generate angles
        weight = {}
        #Each weight varibale is encoded with 4 qubits
        #With the 4 qubits, we can map them into a value
        degrees = 2 * math.pi * np.random.uniform(0, 1, size=(layers[l], layers[l+1], qubit_len))
        weight['sin'] = np.sin(degrees)
        weight['cos'] = np.cos(degrees)
        weight['degree'] = degrees
        weights.append(weight)
    return weights

def initializeBiases(layers):
    if type(layers) is not list:
        print('Wrong input!')
        return None
    layers_num = len(layers)
    biases = []
    for l in range(layers_num-1):
        bias = {}
        #Each bias is encoded in 4 qubits
        #With the 4 qubits, we can map them into a value
        degrees = 2 * math.pi * np.random.uniform(0, 1, size=(layers[l+1], qubit_len))
        bias['sin'] = np.sin(degrees)
        bias['cos'] = np.cos(degrees)
        bias['degrees'] = degrees
        biases.append(bias)
    return biases
        

In [67]:
#layers = [784, 32, 10]
layers = [4, 20, 3]
weights_qubit = initializeWeights(layers)
biases_qubit = initializeBiases(layers)

In [250]:
class feedforwardnetwork:
    '''
    Args:
    input_data: input matrix,[batch_size X feature_num], array
    input_labels: one-hot labels, array
    weights_qubit: define the weights, qubit encoding, list
    biases_qubit: define the biases, qubit_encoding, list
    '''
    def __init__(self, input_data, input_labels, weights_qubit, biases_qubit):
        self.input_data = input_data
        self.input_labels = input_labels
        self.weights_qubit = weights_qubit
        self.biases_qubit = biases_qubit
        
    #Define sigmoid function
    def sigmoid(self, Z):
        return 1/(1 + np.exp(-Z))
    
    #Calculate the accuracy
    def calAccuracy(self, y, y_test):
        '''Calculate Accuracy'''
        return np.mean(y==y_test)
    
    def collapse(self, alpha, beta):
        '''
        Collapse the quantum state into a binary value
        To calculate values later
        Args:
        alpha: the first state
        beta: the second state
        '''
        pick = np.random.uniform(0, 1, alpha.shape)
        #If the random value greater than alpha square, the return true
        #Note this can lead to uncertainties for the results
        states = np.where(pick > alpha**2, 1, 0)
        return states

    
    def binary2decimal(self, states, bound):
        '''
        Map binary values of a variable into a decimal value
        Each parameter can be mapped into binary bits
        Args:
        states: binary bits, like [0 1 1 0]
        bound: the border of the variable
        
        Return: variable values
        '''
        shape = states.shape
        #For weights
        if len(shape) > 2:
            qubit_num = shape[-1]
            values = np.zeros((shape[0], shape[1]))
            for l in np.arange(qubit_num):
                values += states[:, :, l] * (2**l)
                values = -bound + values/(2**qubit_len-1)*2*bound
        #For biases
        else:
            qubit_num = shape[-1]
            values = np.zeros((shape[0]))
            for l in np.arange(qubit_num):
                values += states[:, l] * (2**l)
                values = -bound + values/(2**qubit_len-1)*2*bound
        return values
    
    def quantum2value(self):
        '''
        Map varible qubits into decimal values
        '''
        self.weights = []
        self.weights_bits = []
        for weight_qubit in self.weights_qubit:
            states = self.collapse(weight_qubit['sin'], weight_qubit['cos'])
            values = self.binary2decimal(states, 0.5)
            self.weights.append(values)
            self.weights_bits.append(states)
        self.biases = []
        self.biases_bits = []
        for bias_qubit in self.biases_qubit:
            states = self.collapse(bias_qubit['sin'], bias_qubit['cos'])
            values = self.binary2decimal(states, 0.5)
            self.biases.append(values)
            self.biases_bits.append(states)
        #return self.weights, self.biases
        
    #Create a function to do predictions
    #Map a one-hot vector to a number
    def vec2num(self, data, label_num=10):
        '''Make predictions'''
        if len(data.shape) < 2:
            print('The input has too few dimensions')
            return None
        #Select the class which has largest probability
        #predictions = [(z.argmax()+ 1)%label_num for z in data]
        predictions = data.argmax(axis=1)
        return np.array(predictions)
    
    
    def costFuncWithReg(self, h, lambda1=0.01):
        '''
        Calculate the cost of neural network
        Note here we use cross entropy
        Regularization is also taken into account
        '''
        y = self.input_labels
        if h is None or y is None or self.weights is None:
            print('Invalid Input!')
            return None
        sample_num = len(y)#Length of y
        #Cost of errors
        total = -np.mean(np.sum(y*np.log(h), axis=1))
        #Cost of regularization
        weights = np.array(self.weights)
        reg = 0
        for wgt in weights:
            reg += np.sum(wgt**2) * lambda1/2/sample_num
        total +=  reg    
        return total 
    
    def proceed(self):
        '''
        Finish the procedure of feed forward network 
        and calculate the output
        '''
        self.quantum2value()
        h, output_h, input_z = self.feedforwardNeuralNetwork(self.input_data)
        #labels = self.vec2num(self.input_labels)
        labels = self.input_labels
        predictions = self.vec2num(h)
        #self.fitness = self.costFuncWithReg(h)
        self.accuracy = self.calAccuracy(labels, predictions)
        self.fitness = self.accuracy
        
    def updateWeightBias(self, new_weights_qubit, new_biases_qubit):
        '''
        Update weights and biases
        '''
        self.weights_qubit = new_weights_qubit
        self.biases_qubit = new_biases_qubit
        
    def evaluateTestData(self, new_input_data, new_input_labels):
        '''
        Update data
        '''
        X, y = new_input_data, new_input_labels
        h, output_h, input_z = self.feedforwardNeuralNetwork(X)
        labels = y
        predictions = self.vec2num(h)
        accuracy = self.calAccuracy(labels, predictions)
        return accuracy
    
    #weights: weights for each layer, a list
    def feedforwardNeuralNetwork(self, X):
        '''Calculate feedforward propagation output'''
        ######Deal with extreme cases###
        if X is None or self.weights is None:
            print('Invalid Input!')
            return None
        dim = X.shape
        if len(dim) < 2:
            print('X has too less variables')
            return None

        #####Define variables###########
        layer_num = len(self.weights)
        output_h = []#Output for each layer
        output_h.append(X)#The first layer is equal to input X
        input_z = []#Input for each layer, starts from the second layer
        #####Make alculations for each layer, except the input layer
        for i in range(layer_num):
            z = np.dot(output_h[i], self.weights[i])
            z += self.biases[i]
            h = self.sigmoid(z)
            output_h.append(h)
            input_z.append(z)
        return h, output_h, input_z    

In [353]:
X_train, y_train = mnist.train.images, mnist.train.labels

In [252]:
fw = feedforwardnetwork(X_train, y_train, weights_qubit, biases_qubit)

In [253]:
start = time()
fw.proceed()
end = time()
print('Timing:{:.3f}'.format(end-start))

Timing:0.002


It takes much time to finish computation for each individual. So it must be time-consuming if we have a large population.

In [254]:
fw.fitness

0.375

In [255]:
fw.updateWeightBias(weights_qubit, biases_qubit)

In [256]:
fw.proceed()

In [257]:
fw.fitness

0.36666666666666664

In [258]:
fw.evaluateTestData(X_test, y_test)

0.20000000000000001

Now we have an feedforward object, we can create an instance of neural network by specifying the parameters. We can also create a population of neural networks with different parameters. How to generate the parameters, that's a question we will answer by genetic algorithm.

# 2.Create population

In this project, we can view the parameters of neural networks as genes, our goal is to find optimal genes, namely   parameters that make those neural networks performance well on predictions.
There are four major steps in a genetic algorithm:
- Create Population, randomly create parameters and generate neural networks according to the parameters
- Calculate fitness, calculate the fitness of each individual neural network
- Quantum rotating gates, compared to conventional genetic algorithm, QGA uses quantum rotating gates to update chromosomes.
- Mutate, select certain individual neural networks and make mutations on their chromosomes in terms of NOT gating.

For more information you can read the paper The Improvement of Quantum Genetic Algorithm and Its Application on Function Optimization, written by Huaixiao Wang, Jianyong Liu and etc.

## Create a group of parameters

In [259]:
class initPopulation:
    def __init__(self, layers, pop_size=1000):
        '''
        Args:
        pop_size: number of population, integer
        layers: neuron number for each layer, list
        '''
        self.pop_size = pop_size
        self.layers = layers
        
    def __initializeWeights__(self):
        '''
        Initialize weights between each two neigbouring layers
        '''
        layers = self.layers
        if type(layers) is not list:
            print('Wrong input!')
            return None
        layers_num = len(layers)
        weights = []
        for l in range(layers_num-1):
            #Generate angles
            weight = {}
            #Each weight varibale is encoded with 4 qubits
            #With the 4 qubits, we can map them into a value
            #degrees = 0.25 * np.pi * np.ones((layers[l], layers[l+1], qubit_len))
            degrees = np.ones((layers[l], layers[l+1], qubit_len)) * math.pi/4
            weight['sin'] = np.sin(degrees)
            weight['cos'] = np.cos(degrees)
            weight['degree'] = degrees
            weights.append(weight)
        return weights

    def __initializeBiases__(self):
        '''
        Initialize biases for each layer(except for input layer)
        '''
        layers = self.layers
        if type(layers) is not list:
            print('Wrong input!')
            return None
        layers_num = len(layers)
        biases = []
        for l in range(layers_num-1):
            bias = {}
            #Each bias is encoded in 4 qubits
            #With the 4 qubits, we can map them into a value
            #degrees = 0.25 * np.pi * np.ones((layers[l], layers[l+1], qubit_len))
            degrees = np.ones((layers[l+1], qubit_len)) * math.pi/4
            bias['sin'] = np.sin(degrees)
            bias['cos'] = np.cos(degrees)
            bias['degree'] = degrees
            biases.append(bias)
        return biases
    
    def generatePop(self):
        '''
        Generate a group of weights at random
        '''
        population = []
        for i in range(self.pop_size):
            weights = self.__initializeWeights__()
            biases = self.__initializeBiases__()
            #Initialize an individual
            individual = feedforwardnetwork(X_train, y_train, weights, biases)
            #Calculate fitness
            individual.proceed()
            population.append(individual)
        return population

In [342]:
pop = initPopulation(layers)
population = pop.generatePop()

# 3.Genetic Algorithm

In this part, we are going to realize quantum rotation gates, mutation. 
## Quantum Rotation Gates

Quantum rotation gates is a method to change the probabilities amplitube by shifting the angles which updates the states of parameters, and eventually uodates the values of parameters. We can show the process in a formula below.
Note, we introduce the encoding procedurings above, $x_i$ can be encoded in a qubit string with specific lengths:
$$\left(\begin{array}{ccccc}
		\alpha_{i1} & \alpha_{i2} & \alpha_{i3} & \alpha_{i4} & \alpha_{i5}\\ 
		\beta_{i1} & \beta_{i2} & \beta_{i3} & \beta_{i4} & \beta_{i5}
	\end{array}\right)$$
    
 Suppose the $ith$ original qubit of parameter $x_i$ is $\alpha_{ij}$ and $\beta_{ij}$,
 $$\alpha_{ij} = sin(\theta_{ij}), \ \beta_{ij} = cos(\theta_{ij})$$
 
 And the shift angle is $\Delta \theta_{ij}$, therefore the updated qubit of parameter $x_i$:
 $$\hat \alpha_{ij} = sin(\theta_{ij}-\Delta \theta_{ij}) = sin(\theta_{ij})cos(\Delta \theta_{ij})-cos(\theta_{ij})sin(\Delta \theta_{ij})$$
  $$\hat \beta_{ij} = cos(\theta_{ij}-\Delta \theta_{ij}) = cos(\theta_{ij})cos(\Delta \theta_{ij})+sin(\theta_{ij})sin(\Delta \theta_{ij})$$
 



We can transform the formula into a matrix operation:

$$\left[\begin{array}{ccccc}
		\hat \alpha_{ij} \\ 
		\hat \beta_{ij}
	\end{array}\right] = \left[\begin{array}{ccccc}
		sin(\Delta \theta_{ij}) & -cos(\Delta \theta_{ij})\\ 
		cos(\Delta \theta_{ij}) & sin(\Delta \theta_{ij}) 
	\end{array}\right] \left[\begin{array}{ccccc}
		\alpha_{ij} \\ 
		\beta_{ij}
	\end{array}\right]$$

There are some strategies for rotation gates, here we adopt the one in Huaixiao Wang's paper as below:
![rotation strategy](rotationGateStrategy.png)

In [343]:
import  copy
class GA:
    def __init__(self, population):
        '''
        Initialize genetic algorithm
        '''
        self.population = copy.deepcopy(population)
        self.pop_size = len(population)
        
    
    def rotationMatrix(self, sgn, delta):
        '''
        Calculation the matrix of rotation gate
        Args:
        sgn: sign of the angle rotation direction, +1 or -1
        delta: shift angle of the rotation gate
        '''
        e = sgn *delta
        U = np.array([[np.cos(e), -np.sin(e)], [np.sin(e), np.cos(e)]])
        return U
    
    def rotationAngleDirection(self, bestIndividual, obj):
        '''
        Calculate rotation angles and directions for each individual
        '''
        #Initialize the shift angle
        delta_theta = 0.01 * math.pi
        #Compare the fitness
        fitness_flag = obj.fitness > bestIndividual.fitness
        #Traverse each weight layer
        for j in np.arange(len(obj.weights_qubit)):
            #The jth layer
            #Traverse each parameter in this layer
            #qubit contains alpha and beta
            #Alpha size: layer(i), layer(i+1), bit length
            #A qubit is a pair of alpha and beta
            weight_alpha = obj.weights_qubit[j]['sin']
            #beta size: layer(i), layer(i+1), bit length
            weight_beta = obj.weights_qubit[j]['cos']
            #Original angles:
            degrees = obj.weights_qubit[j]['degree']
            #state size: layer(i), layer(i+1), bit length
            #state, e.g [0 1 1 0]
            weight_bit = obj.weights_bits[j]
            #Traverse each parameter
            best_weight_bit = bestIndividual.weights_bits[j]
            #Search the table
            criteria = (weight_bit + best_weight_bit) == 1
            #Calculate shift angles for each parameter
            delta = criteria * delta_theta
            #Calculate the sign of shift angles
            sgns = np.zeros(weight_bit.shape)#Initialize it with zeros
            #Try to avoid loops, use matrix operation as much as mossible
            #IF xi=0 besti=1, then the difference will be -1
            current_best_bit_flag = weight_bit - best_weight_bit
            #Create a matrix of fitness flag with the same shape as the weight
            fitness_flags = np.ones(weight_bit.shape) * fitness_flag
            #Map 0 into -1
            fitness_flags = np.where(fitness_flags>0, 1, -1)
            alpha_beta_pos = (weight_alpha * weight_beta) > 0
            alpha_beta_neg = (weight_alpha * weight_beta) < 0
            alpha_zero = weight_alpha == 0
            beta_zero = weight_beta == 0
            #if alpha * beta>0
            sgns += current_best_bit_flag * fitness_flags * alpha_beta_pos
            #if alpha * beta<0
            sgns += (-1)*current_best_bit_flag * fitness_flags * alpha_beta_neg
            #if alpha = 0
            #Gnerate +1 -1 at random
            direction = np.random.choice([1, -1], size=weight_bit.shape)
            criteria = current_best_bit_flag * fitness_flags * alpha_zero < 0
            sgns += criteria * direction
            #if beta = 0
            criteria = current_best_bit_flag * fitness_flags * beta_zero > 0
            sgns += criteria * direction
            #Calculate shift angles
            angles = delta * sgns
            #Calculate new angles
            degrees = degrees - angles
            obj.weights_qubit[j]['sin'] = np.sin(degrees)
            obj.weights_qubit[j]['cos'] = np.cos(degrees)
            obj.weights_qubit[j]['degree'] = degrees
        #Update bias
        for j in np.arange(len(obj.biases_qubit)):
            #The jth bias
            #Traverse each parameter in this layer
            #qubit contains alpha and beta
            #Alpha size: layer(i) bit length
            #A qubit is a pair of alpha and beta
            bias_alpha = obj.biases_qubit[j]['sin']
            #beta size: layer(i), layer(i+1), bit length
            bias_beta = obj.biases_qubit[j]['cos']
            #Original angles:
            degrees = obj.biases_qubit[j]['degree']
            #state size: layer(i), layer(i+1), bit length
            #state, e.g [0 1 1 0]
            bias_bit = obj.biases_bits[j]
            #Traverse each parameter
            best_bias_bit = bestIndividual.biases_bits[j]
            #Search the table
            criteria = (bias_bit + best_bias_bit) == 1
            #Calculate shift angles for each parameter
            delta = criteria * delta_theta
            #Calculate the sign of shift angles
            #Shape: bias number * bit length
            sgns = np.zeros(bias_bit.shape)#Initialize it with zeros
            #Try to avoid loops, use matrix operation as much as mossible
            #IF xi=0 besti=1, then the difference will be -1
            current_best_bit_flag = bias_bit - best_bias_bit
            #Create a matrix of fitness flag with the same shape as the weight
            fitness_flags = np.ones(bias_bit.shape) * fitness_flag
            #Map 0 into -1
            fitness_flags = np.where(fitness_flags>0, 1, -1)
            alpha_beta_pos = (bias_alpha * bias_beta) > 0
            alpha_beta_neg = (bias_alpha * bias_beta) < 0
            alpha_zero = bias_alpha == 0
            beta_zero = bias_beta == 0
            #if alpha * beta>0
            sgns += current_best_bit_flag * fitness_flags * alpha_beta_pos
            #if alpha * beta<0
            sgns += (-1)*current_best_bit_flag * fitness_flags * alpha_beta_neg
            #if alpha = 0
            #Gnerate +1 -1 at random
            direction = np.random.choice([1, -1], size=bias_bit.shape)
            criteria = current_best_bit_flag * fitness_flags * alpha_zero < 0
            sgns += criteria * direction
            #if beta = 0
            criteria = current_best_bit_flag * fitness_flags * beta_zero > 0
            sgns += criteria * direction
            #Calculate shift angles
            angles = delta * sgns
            #Calculate new angles
            degrees = degrees - angles
            obj.biases_qubit[j]['sin'] = np.sin(degrees)
            obj.biases_qubit[j]['cos'] = np.cos(degrees)
            obj.biases_qubit[j]['degree'] = degrees
        obj.proceed()
        return obj
    
    def rotatingGates(self, bestIndividual_index):
        '''
        Rotate gates of quantum registra,
        Note, we try to make use of numpy's matrix operations to speed
        Computation
        Args:
        bestIndividual_index: the index of the best individual
        '''
        bestIndividual = self.population[bestIndividual_index]
        indices = [bestIndividual] * self.pop_size
        #self.population = list(map(self.rotationAngleDirection, indices, self.population))
        #Traverse each individual
        for i in np.arange(self.pop_size):
            obj = self.population[i]
            obj = self.rotationAngleDirection(bestIndividual, obj)
            self.population[i] = obj
            
    def NotGates(self, ratio=0.05):
        '''
        Rotate gates of quantum registra,
        Note, we try to make use of numpy's matrix operations to speed
        Computation
        Args:
        bestIndividual_index: the index of the best individual
        '''
        #Traverse each individual
        num = int(self.pop_size * ratio)
        indice = np.random.choice(self.pop_size, num)
        for i in indice:
            obj = self.population[i]
            obj = self.mutation(obj)
            self.population[i] = obj
    
    def mutation(self, obj):
        '''
        Mutation at several random point within an individual
        '''
        #Traverse each weight layer
        for j in np.arange(len(obj.weights_qubit)):
            #The jth layer
            #Traverse each parameter in this layer
            #qubit contains alpha and beta
            #Alpha size: layer(i), layer(i+1), bit length
            #A qubit is a pair of alpha and beta
            weight_alpha = obj.weights_qubit[j]['sin']
            #beta size: layer(i), layer(i+1), bit length
            weight_beta = obj.weights_qubit[j]['cos']
            #Degrees
            degrees = obj.weights_qubit[j]['degree']
            picks = np.random.uniform(0, 1, size=degrees.shape)
            #state size: layer(i), layer(i+1), bit length
            alpha_flag = weight_alpha < picks
            beta_flag = weight_beta < picks
            degrees = degrees - alpha_flag*beta_flag*math.pi/2
            obj.weights_qubit[j]['sin'] = np.sin(degrees)
            obj.weights_qubit[j]['cos'] = np.cos(degrees)
            obj.weights_qubit[j]['degree'] = degrees
        #Update bias
        for j in np.arange(len(obj.biases_qubit)):
            #The jth bias
            #Traverse each parameter in this layer
            #qubit contains alpha and beta
            #Alpha size: layer(i) bit length
            #A qubit is a pair of alpha and beta
            bias_alpha = obj.biases_qubit[j]['sin']
            #beta size: layer(i), layer(i+1), bit length
            bias_beta = obj.biases_qubit[j]['cos']
            #Original angles:
            degrees = obj.biases_qubit[j]['degree']
            picks = np.random.uniform(0, 1, size=degrees.shape)
            alpha_flag = bias_alpha < picks
            beta_flag = bias_beta < picks
            degrees = degrees - alpha_flag*beta_flag*math.pi/2
            #Calculate new angles
            degrees = degrees - alpha_flag*beta_flag*math.pi/2
            obj.biases_qubit[j]['sin'] = np.sin(degrees)
            obj.biases_qubit[j]['cos'] = np.cos(degrees)
            obj.biases_qubit[j]['degree'] = degrees
        obj.proceed()
        return obj
        
    def proceed(self, generation_num = 1):
        '''
        Execute quantum rotation and not gating
        '''
        #Keep the best individual
        best_fitness, optimal_index, _ = self.findMaximalIndividual()
        #Note the variables are referenced, we need to copy in a deep way
        best_individual = copy.deepcopy(self.population[optimal_index])
        for _ in np.arange(generation_num):
            #Quantum Rotation Gate
            print('Best Training Accuracy:', round(best_fitness, 4))
            test_accuracy = best_individual.evaluateTestData(X_test, y_test)
            print('Testing Accuracy:', round(test_accuracy, 4))
            self.rotatingGates(optimal_index)
            #Mutation
            self.NotGates()
            fitness, max_index, min_index = ga.findMaximalIndividual()
            if fitness < best_fitness:
                #Use deep copy or the two objects will be the same actually
                self.population[max_index] = copy.deepcopy(best_individual)
            else:
                best_fitness = fitness
                optimal_index = max_index
                #Note deep copy here
                best_individual = copy.deepcopy(self.population[optimal_index])
        print('Final Training Accuracy:', round(best_fitness, 4))
        return best_individual
        
    def findMaximalIndividual(self):
        fitnesses = np.array([one.fitness for one in self.population])
        optimal_value = max(fitnesses)
        optimal_index = fitnesses.argmax()
        minimal_index = fitnesses.argmin()
        return optimal_value, optimal_index, minimal_index

In [344]:
ga = GA(population)

In [348]:
#Try 20 generations
best = ga.proceed(50)

Best Training Accuracy: 0.825
Testing Accuracy: 0.8333
Best Training Accuracy: 0.825
Testing Accuracy: 0.8333
Best Training Accuracy: 0.825
Testing Accuracy: 0.8333
Best Training Accuracy: 0.825
Testing Accuracy: 0.8333
Best Training Accuracy: 0.825
Testing Accuracy: 0.8333
Best Training Accuracy: 0.825
Testing Accuracy: 0.7333
Best Training Accuracy: 0.825
Testing Accuracy: 0.7333
Best Training Accuracy: 0.825
Testing Accuracy: 0.7333
Best Training Accuracy: 0.825
Testing Accuracy: 0.7333
Best Training Accuracy: 0.825
Testing Accuracy: 0.7333
Best Training Accuracy: 0.825
Testing Accuracy: 0.7333
Best Training Accuracy: 0.825
Testing Accuracy: 0.7333
Best Training Accuracy: 0.9
Testing Accuracy: 0.7667
Best Training Accuracy: 0.9
Testing Accuracy: 0.7667
Best Training Accuracy: 0.9
Testing Accuracy: 0.7667
Best Training Accuracy: 0.9
Testing Accuracy: 0.7667
Best Training Accuracy: 0.9
Testing Accuracy: 0.7667
Best Training Accuracy: 0.9
Testing Accuracy: 0.7667
Best Training Accuracy

In [349]:
best.evaluateTestData(X_train, y_train)

0.95833333333333337

In [350]:
best.evaluateTestData(X_test, y_test)

0.93333333333333335

The final result is competitive compared to backpropagation method.